# Objetivos deste trabalho
- Familiarizar-se com a biblioteca PyTorch
- Definir arquiteturas MLP simples em PyTorch
- Treinar utilizando CIFAR10, testando diferentes arquiteturas, parâmetros, funções de loss e otimizadores
- Comparar os resultados obtidos utilizando apenas Perpceptrons

In [8]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [9]:
# Carregar os datasets

transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
train_loader = DataLoader(dataset=dataset_train, shuffle=True)
test_loader = DataLoader(dataset=dataset_test, shuffle=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [11]:
# Definir a arquitetura MLP

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(32*32, 256)
        self.fc2 = nn.Linear(256, 10)
        self.activation_function = nn.ReLU()
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        return x

In [23]:
# Confere se pode utilizar cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Imprime o dispotivo
print(device)

cpu


In [24]:
model = MLP()
model = model.to(device)
print(model)

MLP(
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
  (activation_function): ReLU()
)


In [25]:
# Definir otimizador e loss
# Nota: testar outros otimizadores e funções de loss (em particular cross entropy)

optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
loss_fn = nn.CrossEntropyLoss()

In [16]:
# Realizar o treinamento aqui

for epoch in range(3):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        # Pega a Entrada e o Label
        input, label = data
        
        input, label = input.to(device), label.to(device)
        
        # Zera os parâmetros dos gradientes
        optimizer.zero_grad()
        
        output = model(input)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Treino Finalizado')


[1,  2000] loss: 1.964
[1,  4000] loss: 1.958
[1,  6000] loss: 1.944
[1,  8000] loss: 1.949
[1, 10000] loss: 1.907
[1, 12000] loss: 1.951
[1, 14000] loss: 1.981
[1, 16000] loss: 1.956
[1, 18000] loss: 1.948
[1, 20000] loss: 1.941
[1, 22000] loss: 1.940
[1, 24000] loss: 1.973
[1, 26000] loss: 1.959
[1, 28000] loss: 1.921
[1, 30000] loss: 1.997
[1, 32000] loss: 1.957
[1, 34000] loss: 1.954
[1, 36000] loss: 1.948
[1, 38000] loss: 1.982
[1, 40000] loss: 1.960
[1, 42000] loss: 1.943
[1, 44000] loss: 1.959
[1, 46000] loss: 1.964
[1, 48000] loss: 1.972
[1, 50000] loss: 1.928
[2,  2000] loss: 1.927
[2,  4000] loss: 1.942
[2,  6000] loss: 1.932
[2,  8000] loss: 1.992
[2, 10000] loss: 1.938
[2, 12000] loss: 1.968
[2, 14000] loss: 1.955
[2, 16000] loss: 1.918
[2, 18000] loss: 1.941
[2, 20000] loss: 1.930
[2, 22000] loss: 1.937
[2, 24000] loss: 1.915
[2, 26000] loss: 1.949
[2, 28000] loss: 1.973
[2, 30000] loss: 1.977
[2, 32000] loss: 1.903
[2, 34000] loss: 1.962
[2, 36000] loss: 1.958
[2, 38000] 

In [21]:
# Avaliar o modelo aqui (no conjunto de teste)

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        image, label = data
        output = model(image)
        _, predicted = torch.max(output.data, 1)
        total += 1 
        correct += (predicted == label).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % 
      ( len(test_loader), 100 * correct / total))


Accuracy of the network on the 10000 test images: 30 %


In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cpu
